this code employ a hybrid feature selection approach combined with machine learning to predict melanoma patients' responses to ICB therapy. The feature selection process integrate two filter methods, univariate ROC-AUC and RliefF, along with one wrapper method, SVM-RFE. then ensemble voting classifier used for evaluate the model accuracy.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, LeaveOneOut, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import xgboost as xgb
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, plot_roc_curve
from yellowbrick.classifier import ROCAUC


# input the data (gene expression array) and labels

In [ ]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]
name

In [ ]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]

# Read logged data
logged_data = pd.read_csv('data/' + name + '_logged_data.csv', index_col='Unnamed: 0').T
print(logged_data.shape)
print(logged_data.head())

# Read labels
label = pd.read_csv('data/' + name + '.Pre.Samples.corr.csv', index_col='Unnamed: 0')
print(label.shape)
print(label.head())

# Count of labels
print(label['Resp_NoResp'].value_counts())

# Normalized count of labels
print(label['Resp_NoResp'].value_counts(normalize=True))

# Label encoding
y = label.replace({'No_Response': 0, 'Response': 1})
labels = ['No_Response', 'Response']  # for plotting convenience later on

print(y.head())
print(y['Resp_NoResp'].head())
print(y.iloc[:, 0].head())
print(y.iloc[:, 0].shape)


In [ ]:
#read the immune related genes from immport
IRG= pd.read_csv('IRG.csv')
print(IRG.shape)
IRG.head()

# data oversampling

In [ ]:
# Define features (X) and target (Y)
X = logged_data
Y = y.iloc[:, 0]

# Oversampling using BorderlineSMOTE
ada = BorderlineSMOTE(
    sampling_strategy='auto',  # samples only the minority class
    random_state=0,  # for reproducibility
    k_neighbors=5,
    m_neighbors=10,
    kind='borderline-1',
    n_jobs=4
)

# Resample the data
X_res, y_res = ada.fit_resample(X, Y)

# Print the shapes of resampled data
print(X_res.shape, y_res.shape)

# Print the counts of the original and resampled target variable
print(Y.value_counts(), y_res.value_counts())


# separate dataset into train and test


In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=0)

# Display shapes of train and test sets
print(X_train.shape, X_test.shape)

# Display first few rows of test set
print(X_test.head())

# Display normalized value counts of target variable in test set
print(y_test.value_counts(normalize=True))

# Keep a copy of the original datasets
X_train_original = X_train.copy()
X_test_original = X_test.copy()

# Check shapes of train and test sets after copying
print(X_train.shape, X_test.shape)


In [ ]:
#X_train = X_train_original
#X_test = X_test_original

In [ ]:
X_train.shape

# Remove constant and quasi-constant features

In [ ]:
# Remove constant features
sel = VarianceThreshold(threshold=0)
sel.fit(X_train)

# Get the number of features that are not constant
num_non_constant_features = sum(sel.get_support())

# Remove quasi-constant features
sel = VarianceThreshold(threshold=0.1)  # 0.1 indicates 99% of observations approximately
sel.fit(X_train)
num_non_quasi_constant_features = sum(sel.get_support())

# Get constant feature names
constant_features = X_train.columns[~sel.get_support()]

# Get non-constant feature names
non_constant_feature_names = X_train.columns[sel.get_support()]

# Transform train and test sets
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

# Convert arrays back to DataFrame with non-constant features
X_train = pd.DataFrame(X_train, index=X_train_original.index, columns=non_constant_feature_names)
X_test = pd.DataFrame(X_test, index=X_test_original.index, columns=non_constant_feature_names)

# Print shapes of transformed data
print(X_train.shape, X_test.shape)

# Display first few rows of transformed data
print(X_train.head())
print(X_test.head())


# Univariate-Performance


In [ ]:
from feature_engine.selection import SelectBySingleFeaturePerformance

# Define the estimator
rf = ExtraTreesClassifier()

# Set up the selector
sel = SelectBySingleFeaturePerformance(
    variables=None,  # Select all features
    estimator=rf,
    scoring="roc_auc",
    cv=3,
    threshold=0.70,
)

# Find predictive features
sel.fit(X_train, y_train)

# Get feature performance
feature_performance = sel.feature_performance_

# Get the number of features to be removed
num_features_to_drop = len(sel.features_to_drop_)

# Remove non-predictive features
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

# Print shapes of transformed data
print(X_train.shape, X_test.shape)


In [ ]:
X_train.head()

In [ ]:
X_train_uni = X_train
X_test_uni = X_test

# relieff

In [ ]:
from skrebate import ReliefF
fs = ReliefF()
fs.fit(X_train.values, y_train.values)


In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()


# Will take some time
for i in range(5,305,5):
    
    indx_sort=fs.feature_importances_.argsort()[-i:][::-1]
    X_train_rf= X_train.iloc[:,indx_sort]
    X_test_rf= X_test.iloc[:,indx_sort]
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard') 
    
   
    print(i)
    
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_rf, y_train)
        y_pred = clf.predict(X_test_rf)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
        
  


    # Number of iterations
    num_iterations = 3

    # Initialize a list to store accuracy scores
    ensemble_scores = []

    # Perform multiple iterations of the VotingClassifier
    for _ in range(num_iterations):
        eclf1.fit(X_train_rf, y_train)
        predictions = eclf1.predict(X_test_rf)
        ensemble_acc = accuracy_score(y_test, predictions)
        ensemble_scores.append(ensemble_acc)

    print("Ensemble Voting Accuracy:", ensemble_scores)
    
    # Calculate the mean accuracy of the ensemble
    mean_ensemble_accuracy = np.mean(ensemble_scores)
    print("Mean Ensemble Voting Accuracy:", mean_ensemble_accuracy)

    print(X_train_rf.columns)
    accuracy_rate.append(mean_ensemble_accuracy)
    

In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(5,300,5),accuracy_rate,color='blue', linestyle='solid', marker='o',
         markerfacecolor='blue', markersize=10)

title =plt.title('ReliefF Accuracy vs. No. of Genes', fontsize=20)
plt.xlabel('No. of Genes', fontsize=16)
plt.ylabel('ReliefF Accuracy', fontsize=16)


In [ ]:
# extract top 95 features ranked by Relieff, as obtainted in the increment feature selection procedure above

In [ ]:
indx_sort=fs.feature_importances_.argsort()[-95:][::-1]
indx_sort

In [ ]:
X_train_rf= X_train.iloc[:,indx_sort]
X_train_rf.shape

In [ ]:
X_test_rf= X_test.iloc[:,indx_sort]
X_test_rf.shape

In [ ]:
# how much immune related genes in the selected genes
IRG_feat=X_train_rf.columns.intersection(IRG['Symbol'])
IRG_feat

In [ ]:
X_train = X_train_rf
X_test= X_test_rf


In [ ]:
X_train.shape

# draw figure 2

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(5,305,5),accuracy_plot['ReliefF_accuracy'],color='blue', linestyle='solid', marker='o',
         markerfacecolor='blue', markersize=10)


title =plt.title('Uni + ReliefF Accuracy vs. No. of Genes', fontsize=20)
plt.xlabel('No. of Genes', fontsize=16)
plt.ylabel('ReliefF Accuracy', fontsize=16)
#plt.legend()
plt.savefig('gide_accuracy_plot_uni relieff.jpg')
plt.savefig('gide_accuracy_plot_uni relieff.eps',dpi=300)
plt.savefig('gide_accuracy_plot_uni relieff.pdf',dpi=300)

# SVM-RFE

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()



# Will take some time
for i in range(1, 51, 1):
    
    svm= SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
    sel_ = RFE(svm, n_features_to_select=i)
    sel_.fit(X_train, y_train)
    selected_feat = X_train.columns[(sel_.get_support())]
    print(len(selected_feat))
    X_train_rf = X_train[selected_feat]
    X_test_rf = X_test[selected_feat]
    # display features
    
    
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard')    
    print(i)
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_rf, y_train)
        y_pred = clf.predict(X_test_rf)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    
    eclf1.fit(X_train_rf, y_train)
    predictions = eclf1.predict(X_test_rf)
    print(classification_report(y_test, predictions))
    print(X_train_rf.columns)
    accuracy_rate.append(round(accuracy_score(y_test, predictions),3))


In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(1,51,1),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('uni+ relieff + SVM-RFE')
plt.xlabel('No. of Genes')
plt.ylabel('Accuracy')

In [ ]:
# extract the best set of genes by SVM-RFE, as obtainted in the increment feature selection procedure above.

In [ ]:

from sklearn.feature_selection import RFE
sel_ = RFE(SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear'), n_features_to_select=35)
sel_.fit(X_train, y_train)

In [ ]:
selected_feat = X_train.columns[(sel_.get_support())]
len(selected_feat)

In [ ]:
X_train_rf = X_train[selected_feat]
X_test_rf = X_test[selected_feat]
X_train_rf.shape, X_test_rf.shape

In [ ]:
X_train_rf.columns

In [ ]:
X_train = X_train_rf
X_test= X_test_rf


In [ ]:
# the reduced data with the final selected genes only

In [ ]:
X_train_selected = pd.DataFrame(X_train_original,index= X_train_original.index, columns= X_train_rf.columns)
X_train_selected.head()

In [ ]:
X_test_selected = pd.DataFrame(X_test_original,index= X_test_original.index, columns= X_train_rf.columns)
X_test_selected.head()

In [ ]:
X_train =X_train_selected
X_test = X_test_selected

# performance evaluation

In [ ]:
from sklearn.ensemble import VotingClassifier
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()            
               


df_test = pd.DataFrame()
df_test['y_test'] = y_test
for clf in (clf1,clf2, clf3, clf4, clf5):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    df_test[clf.__class__.__name__] = round(accuracy_score(y_test, y_pred),3)
    print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    



In [ ]:

clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()            
               
eclf1 = VotingClassifier(estimators=[  ('logistic', clf2),('SVM', clf3),
                                        ('XGB', clf5), ('extratree',clf1), 
                                        ('AdaBoost', clf4)
                                        ], voting='hard') 




for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))



eclf1.fit(X_train, y_train)
predictions = eclf1.predict(X_test)
print(classification_report(y_test, predictions))

confusion_mat = confusion_matrix(y_test, predictions)
# Extract values from the confusion matrix
true_positive = confusion_mat[1, 1]
false_positive = confusion_mat[0, 1]
true_negative = confusion_mat[0, 0]
false_negative = confusion_mat[1, 0]

# Calculate Sensitivity (True Positive Rate)
sensitivity = true_positive / (true_positive + false_negative)

# Calculate Specificity
specificity = true_negative / (true_negative + false_positive)

# Print the results
print(f'Sensitivity (True Positive Rate): {sensitivity:.2f}')
print(f'Specificity: {specificity:.2f}')
